In [3]:
import os

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Langsmith Tracking And Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [5]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Data validation using pydantic
class ProductDetails(BaseModel):
    productName: str = Field(description="Name of the product")
    productDeatail: str = Field(description="Basic product description")
    price: float = Field(description="Price of the product in USD")



In [7]:
# Prompt
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=ProductDetails)

format_instructions = parser.get_format_instructions()


prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are an assistant in amazon website, based on the user query extract the product details and return in JSON format. Strictly follow the format instructions. \n{format_instructions}, Remeber the product price should be in USD(attach it with price)'),
        ('user','{input}')
    ]
)

In [ ]:
model = ChatOpenAI(model='gpt-4.1', temperature=0.7)

In [14]:
chain = prompt|model|parser

In [15]:
result = chain.invoke({'input':'Can u give me the product details of Macbook Air','format_instructions':format_instructions})

In [16]:
print(result)

{'productName': 'MacBook Air', 'productDeatail': "The MacBook Air is Apple's ultra-thin, lightweight laptop featuring a Retina display, Apple M-series chip for high performance and efficiency, all-day battery life, and a sleek aluminum chassis. It is ideal for everyday tasks, productivity, and portability.", 'price': 999}


In [17]:
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"productName": {"description": "Name of the product", "title": "Productname", "type": "string"}, "productDeatail": {"description": "Basic product description", "title": "Productdeatail", "type": "string"}, "price": {"description": "Price of the product in USD", "title": "Price", "type": "number"}}, "required": ["productName", "productDeatail", "price"]}\n```'